In [34]:
import numpy as np
import pandas as pd
from os import listdir
from os.path import isfile, join

folders = ["Shanghai1/","Shanghai2/","Shanghai3/","Beijing/"]
data = pd.DataFrame(columns=['filename', 'datetime'])

for folder in folders:
    files = listdir('../'+folder)
    df_tmp = pd.DataFrame(columns=['filename', 'datetime'])
    df_tmp['filename'] = [(folder+file[:-4]) for file in files]
    df_tmp['datetime'] = [((file[:-4]).replace("_","")) for file in files]
    data = data.append(df_tmp,ignore_index=True)

print(len(data))
data.head()

2281


,filename,datetime
0,Shanghai1/201407241115,201407241115
1,Shanghai1/201407271015,201407271015
2,Shanghai1/201407121315,201407121315
3,Shanghai1/201406171145,201406171145
4,Shanghai1/201407191415,201407191415


In [37]:
# parse datetime
import datetime

for index,row in data.iterrows():
    dt_tmp = row["datetime"]
    if dt_tmp == ".DSS":
        print(index)
        continue
    data.set_value(index,"dt_round",datetime.datetime.strptime(dt_tmp,'%Y%m%d%H%M'))

#data['dt_round'] = pd.to_datetime(data['datetime'],format="%Y%m%d%H%M")
data.head()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  if __name__ == '__main__':


1724


,filename,datetime,dt_round
0,Shanghai1/201407241115,201407241115,2014-07-24 11:15:00
1,Shanghai1/201407271015,201407271015,2014-07-27 10:15:00
2,Shanghai1/201407121315,201407121315,2014-07-12 13:15:00
3,Shanghai1/201406171145,201406171145,2014-06-17 11:45:00
4,Shanghai1/201407191415,201407191415,2014-07-19 14:15:00


In [39]:
data.drop([1724],inplace=True)
len(data)

2280

In [46]:
pm_sh = pd.read_csv('../shanghai-2014.csv')
pm_b = pd.read_csv('../beijing-2014.csv')

pm_sh["Date (LST)"] = pd.to_datetime(pm_sh["Date (LST)"])
pm_b["Date (LST)"] = pd.to_datetime(pm_b["Date (LST)"])
pm_sh.head()

,Site,Parameter,Date (LST),Year,Month,Day,Hour,Value,Duration,QC Name
0,Shanghai,PM2.5,2014-01-01 00:00:00,2014,1,1,0,136,1 Hr,Valid
1,Shanghai,PM2.5,2014-01-01 01:00:00,2014,1,1,1,173,1 Hr,Valid
2,Shanghai,PM2.5,2014-01-01 02:00:00,2014,1,1,2,166,1 Hr,Valid
3,Shanghai,PM2.5,2014-01-01 03:00:00,2014,1,1,3,157,1 Hr,Valid
4,Shanghai,PM2.5,2014-01-01 04:00:00,2014,1,1,4,157,1 Hr,Valid


In [47]:
pm_b.head()

,Site,Parameter,Date (LST),Year,Month,Day,Hour,Value,Duration,QC Name
0,Beijing,PM2.5,2014-01-01 00:00:00,2014,1,1,0,53,1 Hr,Valid
1,Beijing,PM2.5,2014-01-01 01:00:00,2014,1,1,1,65,1 Hr,Valid
2,Beijing,PM2.5,2014-01-01 02:00:00,2014,1,1,2,70,1 Hr,Valid
3,Beijing,PM2.5,2014-01-01 03:00:00,2014,1,1,3,79,1 Hr,Valid
4,Beijing,PM2.5,2014-01-01 04:00:00,2014,1,1,4,92,1 Hr,Valid


In [48]:
data["dt_round_2"] = data["dt_round"].dt.round('60min')
data.head()

,filename,datetime,dt_round,dt_round_2
0,Shanghai1/201407241115,201407241115,2014-07-24 11:15:00,2014-07-24 11:00:00
1,Shanghai1/201407271015,201407271015,2014-07-27 10:15:00,2014-07-27 10:00:00
2,Shanghai1/201407121315,201407121315,2014-07-12 13:15:00,2014-07-12 13:00:00
3,Shanghai1/201406171145,201406171145,2014-06-17 11:45:00,2014-06-17 12:00:00
4,Shanghai1/201407191415,201407191415,2014-07-19 14:15:00,2014-07-19 14:00:00


In [59]:
data["ppm"] = None
drop_idx = []
for index, row in data.iterrows():
    place = row["filename"][0]
    pm = None
    if place == "S":
        pm = pm_sh.loc[pm_sh['Date (LST)'] == row['dt_round_2']]
        if pm.iloc[0]['QC Name'] == "Valid":
            pm = float(pm.iloc[0]['Value'])
        else:
            drop_idx.append(index)
            continue
    else:
        pm = pm_b.loc[pm_b['Date (LST)'] == row['dt_round_2']]
        if pm.iloc[0]['QC Name'] == "Valid":
            pm = float(pm.iloc[0]['Value'])
        else:
            drop_idx.append(index)
            continue
    data.set_value(index,"ppm",pm)

print(len(drop_idx))
data.tail()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:20: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


51


,filename,datetime,dt_round,dt_round_2,ppm
2276,Beijing/2014_11_27_0732,201411270732,2014-11-27 07:32:00,2014-11-27 08:00:00,14
2277,Beijing/2014_12_27_0806,201412270806,2014-12-27 08:06:00,2014-12-27 08:00:00,182
2278,Beijing/2014_03_05_0710,201403050710,2014-03-05 07:10:00,2014-03-05 07:00:00,35
2279,Beijing/2014_03_18_0701,201403180701,2014-03-18 07:01:00,2014-03-18 07:00:00,20
2280,Beijing/2014_08_02_0623,201408020623,2014-08-02 06:23:00,2014-08-02 06:00:00,62


In [61]:
print(len(data))
data.drop(drop_idx,inplace=True)
print(len(data))

2280
2229


In [62]:
data.to_csv('../single_data.csv',index=False)